In [25]:
import pyAgrum as gum
import pyAgrum.lib.notebook
import numpy as np

# 1. Initialize the Bayesian Network
bn = gum.BayesNet('SurfingConditionsBN')

# 2. Adding Chance Nodes
pressure = bn.add(gum.LabelizedVariable("Pressure", "Pressure", 2))          # States: Low, High
wind_speed = bn.add(gum.LabelizedVariable("WindSpeed", "Wind Speed", 3))    # States: Low, Medium, High
wind_direction = bn.add(gum.LabelizedVariable("WindDirection", "Wind Direction", 2))  # States: Offshore, Onshore
wind_conditions = bn.add(gum.LabelizedVariable("WindConditions", "Wind Conditions", 3))  # States: Good, Okay, Bad
swell_direction = bn.add(gum.LabelizedVariable("SwellDirection", "Swell Direction", 2))  # States: Protected, Not Protected
wave_period = bn.add(gum.LabelizedVariable("WavePeriod", "Wave Period", 3))  # States: <8, 8-12, >12
wave_height = bn.add(gum.LabelizedVariable("WaveHeight", "Wave Height", 3))  # States: Small, Medium, Big
actual_wave_size = bn.add(gum.LabelizedVariable("ActualWaveSize", "Actual Wave Size", 3))  # States: Small, Medium, Big

# 3. Adding Utility Node
utility = bn.add(gum.LabelizedVariable("Utility", "Utility", 1))  # Utility nodes have a single state

# 4. Defining the Network Structure (Arcs)
bn.addArc(pressure, wind_speed)
bn.addArc(pressure, wind_direction)
bn.addArc(wind_speed, wind_conditions)
bn.addArc(wind_direction, wind_conditions)
bn.addArc(swell_direction, actual_wave_size)
bn.addArc(wave_height, actual_wave_size)
bn.addArc(wave_period, wave_height)

# Connecting Utility Node to Relevant Chance Nodes
bn.addArc(actual_wave_size, utility)
bn.addArc(wind_conditions, utility)

# 5. Defining Conditional Probability Distributions (CPDs) for Chance Nodes

# Pressure has no parents
bn.cpt(pressure).fillWith([0.5, 0.5])  # Low, High

# WindSpeed depends on Pressure
bn.cpt(wind_speed).fillWith([
    # Pressure=Low
    0.6, 0.3, 0.1,  # Low: Low, Medium, High Wind Speed
    # Pressure=High
    0.2, 0.5, 0.3   # High: Low, Medium, High Wind Speed
])

# WindDirection depends on Pressure
bn.cpt(wind_direction).fillWith([
    # Pressure=Low
    0.7, 0.3,        # Offshore, Onshore
    # Pressure=High
    0.4, 0.6         # Offshore, Onshore
])

# WindConditions depends on WindSpeed and WindDirection
bn.cpt(wind_conditions).fillWith([
    # WindSpeed=Low, WindDirection=Offshore
    0.7, 0.2, 0.1,    # Good, Okay, Bad
    # WindSpeed=Low, WindDirection=Onshore
    0.6, 0.3, 0.1,    # Good, Okay, Bad
    # WindSpeed=Medium, WindDirection=Offshore
    0.5, 0.3, 0.2,    # Good, Okay, Bad
    # WindSpeed=Medium, WindDirection=Onshore
    0.4, 0.4, 0.2,    # Good, Okay, Bad
    # WindSpeed=High, WindDirection=Offshore
    0.3, 0.3, 0.4,    # Good, Okay, Bad
    # WindSpeed=High, WindDirection=Onshore
    0.2, 0.3, 0.5     # Good, Okay, Bad
])

# SwellDirection has no parents
bn.cpt(swell_direction).fillWith([0.6, 0.4])  # Protected, Not Protected

# WavePeriod has no parents
bn.cpt(wave_period).fillWith([0.3, 0.4, 0.3])  # <8, 8-12, >12

# WaveHeight depends on WavePeriod
bn.cpt(wave_height).fillWith([
    # WavePeriod=<8
    0.7, 0.2, 0.1,  # Small, Medium, Big
    # WavePeriod=8-12
    0.4, 0.4, 0.2,  # Small, Medium, Big
    # WavePeriod=>12
    0.2, 0.3, 0.5   # Small, Medium, Big
])

# ActualWaveSize depends on SwellDirection and WaveHeight
bn.cpt(actual_wave_size).fillWith([
    # SwellDirection=Protected, WaveHeight=Small
    0.8, 0.15, 0.05,  # Small, Medium, Big
    # SwellDirection=Protected, WaveHeight=Medium
    0.3, 0.5, 0.2,    # Small, Medium, Big
    # SwellDirection=Protected, WaveHeight=Big
    0.1, 0.4, 0.5,    # Small, Medium, Big
    # SwellDirection=Not Protected, WaveHeight=Small
    0.7, 0.2, 0.1,    # Small, Medium, Big
    # SwellDirection=Not Protected, WaveHeight=Medium
    0.2, 0.5, 0.3,    # Small, Medium, Big
    # SwellDirection=Not Protected, WaveHeight=Big
    0.05, 0.25, 0.7    # Small, Medium, Big
])


(pyAgrum.Potential<double>@000002499089EE50) 
             ||  ActualWaveSize             |
SwellD|WaveHe||0        |1        |2        |
------|------||---------|---------|---------|
0     |0     || 0.8000  | 0.1500  | 0.0500  |
1     |0     || 0.3000  | 0.5000  | 0.2000  |
0     |1     || 0.1000  | 0.4000  | 0.5000  |
1     |1     || 0.7000  | 0.2000  | 0.1000  |
0     |2     || 0.2000  | 0.5000  | 0.3000  |
1     |2     || 0.0500  | 0.2500  | 0.7000  |

In [26]:
bn

(pyAgrum.BayesNet<double>@0000024990F74520) BN{nodes: 9, arcs: 9, domainSize: 1944, dim: 40, mem: 568o}